In [ ]:
!gdown -q 1N_XeCxVWjnh67Wr-N1Ca21YUcALU-8G6
!unzip -q TruckDataset.zip

In [ ]:
import os
import cv2
import tensorflow as tf
import numpy as np
import random

In [ ]:
class CustomDataGen(tf.keras.utils.Sequence):
    def __init__(self, imgs_folder, labels_txt, batch_size):
        self.batch_size = batch_size

        self.dataset = []
        with open(labels_txt, 'r') as f:
            for i in f.readlines():
                img_file, label = i.split(",")
                img_file = os.path.join(imgs_folder, img_file)
                label = int(label)
                self.dataset.append((img_file, label))

        random.shuffle(self.dataset)

    def __len__(self):
        return len(self.dataset) // self.batch_size


    def get_data(self, batch):
        X = np.array([cv2.imread(x) for x, _ in batch]) / 255.0
        Y = np.array([y for _, y in batch])
        return X, Y

    def __getitem__(self, index):
        begin = self.batch_size * index
        end   = self.batch_size * (index + 1)
        batch = self.dataset[begin : end]
        return self.get_data(batch)

In [ ]:
train_generator = CustomDataGen("/content/Dataset/train/images", "/content/Dataset/train/number_of_axles_train.txt", 16)
valid_generator = CustomDataGen("/content/Dataset/valid/images", "/content/Dataset/valid/number_of_axles_valid.txt", 16)
test_generator = CustomDataGen("/content/Dataset/test/images", "/content/Dataset/test/number_of_axles_test.txt", 16)

In [ ]:
test_generator.__getitem__(0)[0].shape

(16, 640, 640, 3)

In [ ]:
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, GlobalAveragePooling2D, BatchNormalization

def create_model():
    base_model = EfficientNetB3(include_top=False, weights='imagenet')
    for layer in base_model.layers:
        layer.trainable = True

    x = input_layer = Input(shape=(640, 640, 3))
    x = base_model(x)
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dense(32, activation='relu')(x)
    output = Dense(1)(x)

    model = Model(inputs=input_layer, outputs=output)

    return model

model = create_model()
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)           │ (None, 640, 640, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ efficientnetb3 (Functional)          │ (None, 20, 20, 1536)        │      10,783,535 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1536)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 512)                 │         786,944 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 32)                  │           8,224 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,710,064 (44.67 MB)

 Trainable params: 11,622,761 (44.34 MB)

 Non-trainable params: 87,303 (341.03 KB)

In [ ]:
from tensorflow.keras.optimizers import SGD, Adam

optimizer = Adam(learning_rate = 0.001)
model.compile(
    optimizer = optimizer,
    loss = "mse",
    metrics=['mae']
)

es = tf.keras.callbacks.EarlyStopping( monitor='val_loss', mode='min', restore_best_weights=True, patience=3)


history = model.fit(
    train_generator,
    epochs = 10,
    validation_data = valid_generator,
    callbacks = [es]
)

Epoch 1/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 231s 1s/step - loss: 3.9734 - mae: 1.4429 - val_loss: 3.7994 - val_mae: 1.7807
Epoch 2/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 102s 1s/step - loss: 0.5049 - mae: 0.5282 - val_loss: 3.3250 - val_mae: 1.5611
Epoch 3/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 141s 1s/step - loss: 0.2919 - mae: 0.3998 - val_loss: 3.5852 - val_mae: 1.4288
Epoch 4/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 142s 1s/step - loss: 0.1301 - mae: 0.2650 - val_loss: 1.5190 - val_mae: 0.9742
Epoch 5/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 142s 1s/step - loss: 0.0956 - mae: 0.2308 - val_loss: 0.7929 - val_mae: 0.6944
Epoch 6/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 100s 1s/step - loss: 0.1193 - mae: 0.2584 - val_loss: 0.5761 - val_mae: 0.5227
Epoch 7/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 101s 1s/step - loss: 0.0917 - mae: 0.2211 - val_loss: 0.2342 - val_mae: 0.3033
Epoch 8/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 100s 1s/step - loss: 0.0807 - mae: 0.2135 - val_loss: 0.1638 - val_mae: 0.2468
Epoch 9/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 142s 1s/step - loss: 0.063

In [ ]:
model.evaluate(test_generator)

6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 286ms/step - loss: 0.1700 - mae: 0.1910


[0.11584365367889404, 0.17271727323532104]

In [ ]:
img = cv2.imread("/content/Dataset/test/images/GOPR6698_color-ROI-1-183_jpg.rf.6562d5af9c1814430f170387294177ab.jpg") / 255.
img = np.array([img])

predicao = model.predict(img)[0][0]
round(predicao)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step


5

# Avaliando o modelo

In [ ]:
from sklearn.metrics import confusion_matrix

y_true = []
y_pred = []

with open("/content/Dataset/test/number_of_axles_test.txt", 'r') as f:
    for i in f.readlines():
        img_file, label = i.split(",")
        y_true.append(int(label))

        img_file = os.path.join("/content/Dataset/test/images", img_file)
        img = cv2.imread(img_file) / 255.
        img = np.array([img])

        predicao = round(model.predict(img, verbose = 0)[0][0])
        if predicao != int(label):
            print(f"{int(label)}: {predicao}")
        y_pred.append(predicao)



7: 6
3: 2
4: 3
5: 6
6: 5
1: 3
5: 6
4: 3
4: 3


In [ ]:
print(y_true)

[7, 3, 5, 5, 3, 3, 3, 3, 3, 2, 2, 3, 2, 5, 3, 2, 3, 3, 3, 2, 3, 6, 3, 6, 5, 2, 3, 7, 3, 2, 3, 2, 2, 2, 2, 3, 3, 3, 6, 6, 2, 3, 3, 2, 3, 4, 6, 6, 3, 7, 6, 3, 2, 3, 5, 6, 2, 5, 3, 7, 3, 2, 5, 5, 6, 6, 2, 2, 5, 6, 2, 1, 2, 2, 3, 3, 4, 4, 7, 3, 3, 3, 3, 3, 5, 7, 3, 3, 3, 2, 2, 3, 3, 3, 2, 3, 6, 5, 4, 3, 7, 2, 3, 4, 5, 3, 6, 3, 3]


In [ ]:
confusion_matrix(y_true, y_pred)

array([[ 0,  0,  1,  0,  0,  0,  0],
       [ 0, 25,  0,  0,  0,  0,  0],
       [ 0,  1, 45,  0,  0,  0,  0],
       [ 0,  0,  3,  2,  0,  0,  0],
       [ 0,  0,  0,  0, 10,  2,  0],
       [ 0,  0,  0,  0,  1, 12,  0],
       [ 0,  0,  0,  0,  0,  1,  6]])

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(y_true, y_pred, average="macro")

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


(0.7984281025097352, 0.7131162833026187, 0.7355677352792904, None)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_true, y_pred)

0.9174311926605505